## Configuring Jupyter notebook

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.18.2`
import $ivy.`edu.holycross.shot.cite::xcite:4.2.0`
import $ivy.`edu.holycross.shot::midvalidator:10.0.0`
import $ivy.`edu.holycross.shot::latincorpus:2.2.1`
import $ivy.`edu.holycross.shot::latphone:2.7.2`

Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ohco2_2.12/10.18.2/ohco2_2.12-10.18.2.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ohco2_2.12/10.18.2/ohco2_2.12-10.18.2.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/ohco2_2.12/10.18.2/ohco2_2.12-10.18.2.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.0.6/scala-xml_2.12-1.0.6.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/seqcomp_2.12/1.0.0/seqcomp_2.12-1.0.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.1.1/xcite_2.12-4.1.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/seqcomp_2.12/1.0.0/seqcomp_2.12-1.0.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/s

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                        

import $ivy.$                                      

import $ivy.$                                   

## Load corpus from URL

In [3]:
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._

val hyginusUrl = "https://raw.githubusercontent.com/neelsmith/hctexts/master/cex/hyginus.cex"

val corpus = CorpusSource.fromUrl(hyginusUrl, cexHeader = true)

import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._


hyginusUrl: String = "https://raw.githubusercontent.com/neelsmith/hctexts/master/cex/hyginus.cex"
corpus: Corpus = Corpus(
  Vector(
    CitableNode(
      CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:t.1"),
      "EXCERPTA EX HYGINI GENEALOGIIS, VOLGO FABVLAE."
    ),
    CitableNode(
      CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:pr.1"),
      "Ex Caligine Chaos: ex Chao et Caligine Nox Dies Erebus Aether. ex Nocte et Erebo Fatum Senectus Mors Letum Continentia Somnus Somnia Amor id est Lysimeles, Epiphron dumiles Porphyrion Epaphus Discordia Miseria Petulantia Nemesis Euphrosyne Amicitia Misericordia Styx; Parcae tres, id est Clotho Lachesis Atropos; Hesperides, Aegle Hesperie aerica."
    ),
    CitableNode(
      CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:pr.2"),
      "Ex Aethere et Die Terra Caelum Mare."
    ),
    CitableNode(
      CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:pr.3"),
      "Ex Aethere

## Create tokenizable corpus

Load FST parser output.

In [4]:
val hyginusFstUrl = "https://raw.githubusercontent.com/neelsmith/hctexts/master/parser-output/hyginus/hyginus-parses.txt"
import scala.io.Source
val fstOutput = Source.fromURL(hyginusFstUrl).getLines.toVector

hyginusFstUrl: String = "https://raw.githubusercontent.com/neelsmith/hctexts/master/parser-output/hyginus/hyginus-parses.txt"
import scala.io.Source

fstOutput: Vector[String] = Vector(
  "> et",
  "<u>latcommon.indecln16278</u><u>ls.n16278</u>et<indecl><indeclconj><div><indeclconj><indecl><u>indeclinfl.2</u>",
  "> in",
  "<u>latcommon.indecln22111</u><u>ls.n22111</u>in<indecl><indeclprep><div><indeclprep><indecl><u>indeclinfl.1</u>",
  "> cum",
  "<u>latcommon.indecln11872</u><u>ls.n11872</u>cum<indecl><indeclprep><div><indeclprep><indecl><u>indeclinfl.1</u>",
  "> filius",
  "<u>latcommon.nounn18185</u><u>ls.n18185</u>fili<noun><masc><us_i><div><us_i><noun>us<masc><nom><sg><u>livymorph.us_i1</u>",
  "> ex",
  "<u>latcommon.indecln16519</u><u>ls.n16519</u>ex<indecl><indeclconj><div><indeclconj><indecl><u>indeclinfl.2</u>",
  "> est",
  "<u>latcommon.irregverbn46529c</u><u>ls.n46529</u><#>est<3rd><sg><pres><indic><act><irregcverb><div><irregcverb><u>irreginfl.3</u>",
  "> ad",
  "<u>l

Tokenize corpus according to its orthographic system (here, `Latin23Alphabet`).

In [5]:
import edu.holycross.shot.latin._

import edu.holycross.shot.mid.validator._


val tcorpus = TokenizableCorpus(corpus, Latin23Alphabet )
val wordList =  tcorpus.wordList

import edu.holycross.shot.latin._


import edu.holycross.shot.mid.validator._



tcorpus: TokenizableCorpus = TokenizableCorpus(
  Corpus(
    Vector(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:t.1"),
        "EXCERPTA EX HYGINI GENEALOGIIS, VOLGO FABVLAE."
      ),
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:pr.1"),
        "Ex Caligine Chaos: ex Chao et Caligine Nox Dies Erebus Aether. ex Nocte et Erebo Fatum Senectus Mors Letum Continentia Somnus Somnia Amor id est Lysimeles, Epiphron dumiles Porphyrion Epaphus Discordia Miseria Petulantia Nemesis Euphrosyne Amicitia Misericordia Styx; Parcae tres, id est Clotho Lachesis Atropos; Hesperides, Aegle Hesperie aerica."
      ),
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:pr.2"),
        "Ex Aethere et Die Terra Caelum Mare."
      ),
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc:pr.3"),
        "Ex Aethere et Terra Dolor Dolus 

Combine parser output with tokenized corpus to get a `LatinCorpus` instance.

In [6]:
import edu.holycross.shot.latincorpus._

val lc = LatinCorpus.fromFstLines(
      corpus,
       Latin23Alphabet,
     fstOutput,
      strict = false
    )


2020-02-26 13:25:23.244Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.4,VOLGO)  - (LatinCorpus.scala:221)
2020-02-26 13:25:23.296Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.5,FABVLAE)  - (LatinCorpus.scala:221)
2020-02-26 13:25:23.348Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:pr.3.10,Vltio)  - (LatinCorpus.scala:221)
2020-02-26 13:25:23.438Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:pr.9.8,(pro)  - (LatinCorpus.scala:221)
2020-02-26 13:25:23.460Z  warn [LatinCorpus] 
Failed on token category opt None.  Citable node was 
	CitableNode(urn:cts:latinLit:stoa1263.stoa001.hc_tkns:pr.10.7,.)  - (LatinCorpus.scala:221)
2020-02-26 13:25:23.461

import edu.holycross.shot.latincorpus._


lc: LatinCorpus = LatinCorpus(
  Vector(
    LatinToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.0"),
        "EXCERPTA"
      ),
      LexicalToken,
      Vector()
    ),
    LatinToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.1"),
        "EX"
      ),
      LexicalToken,
      Vector()
    ),
    LatinToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.2"),
        "HYGINI"
      ),
      LexicalToken,
      Vector()
    ),
    LatinToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.3"),
        "GENEALOGIIS"
      ),
      LexicalToken,
      Vector()
    ),
    LatinToken(
      CitableNode(
        CtsUrn("urn:cts:latinLit:stoa1263.stoa001.hc_tkns:t.1.3_0"),
        ","
      ),
...

In [7]:
// This should be the number of distinct analyzed tokens
lc.lexemeTokenIndex.size

res6: Int = 2834

In [8]:
// This is the histogram of recognized lexemes:
lc.labelledLexemeHistogram

res7: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
    Frequency("ls.n40103:qui1", 1136),
    Frequency("ls.n16278:et", 935),
    Frequency("ls.n40242:quis1", 834),
    Frequency("ls.n25029:is", 798),
    Frequency("ls.n46529:sum1", 756),
    Frequency("ls.n22111:in1", 707),
    Frequency("ls.n18185:filius", 683),
    Frequency("ls.n11872:cum1", 636),
    Frequency("ls.n16519:ex", 402),
    Frequency("ls.n18173:filia", 354),
    Frequency("ls.n46498:sui", 350),
    Frequency("ls.n665:ad", 298),
    Frequency("ls.n47174:suus", 241),
    Frequency("ls.n49975:ut", 219),
    Frequency("ls.n15868:eo1", 217),
    Frequency("ls.n13804:dico2", 198),
    Frequency("ls.n24257:interficio", 187),
    Frequency("ls.n5:Aba", 171),
    Frequency("ls.n20640:hic", 161),
    Frequency("ls.n50442:venio", 156),
    Frequency("ls.n4:ab", 153),
    Frequency("ls.n17516:facio", 151),
    Frequency("ls.n41625:rex1", 146),
    Frequency("ls.n4633:autem", 136),
    Frequency("ls.n14599

In [9]:
// It would be nice to visualize, so let's use the 
// plotly library with ammonite sh:
// Make plotly libraries available to this notebook:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-almond_2.12/0.7.1/plotly-almond_2.12-0.7.1.pom
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-render_2.12/0.7.1/plotly-render_2.12-0.7.1.pom
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-core_2.12/0.7.1/plotly-core_2.12-0.7.1.pom
Downloaded https://repo1.maven.org/maven2/org/webjars/bower/plotly.js/1.41.3/plotly.js-1.41.3.pom
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-core_2.12/0.7.1/plotly-core_2.12-0.7.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/webjars/bower/plotly.js/1.41.3/plotly.js-1.41.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-almond_2.12/0.7.1/plotly-almond_2.12-0.7.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-render_2.12/0.7.1/plotly-render_2.12-0.7.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/plotly-scala/plotly-almond_2.12/0.7.1/plotly-almond_2.12

import $ivy.$                                      

In [10]:
// Import plotly libraries, and set display defaults suggested for use in Jupyter NBs:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


## Zipf's Law for analyzed lexemes in Hyginus

In [11]:
val items = lc.labelledLexemeHistogram.frequencies.map(fr => fr.item)
val counts = lc.labelledLexemeHistogram.frequencies.map(fr => fr.count)
lc.labelledLexemeHistogram
val zipf = Vector(
  Bar(x = items, y = counts)
)
plot(zipf)

items: Vector[String] = Vector(
  "ls.n40103:qui1",
...
counts: Vector[Int] = Vector(
  1136,
...
res10_2: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
...
zipf: Vector[Bar] = Vector(
  Bar(
...
res10_4: String = "plot-65393ab5-c537-49ec-8a98-fa99264b2fca"

## Zipf's Law for analyzed tokens in Hyginus

In [12]:
val items = lc.tcorpus.lexHistogram.frequencies.map(fr => fr.item)
val counts = lc.tcorpus.lexHistogram.frequencies.map(fr => fr.count)
lc.labelledLexemeHistogram
val zipfTokens = Vector(
  Bar(x = items, y = counts)
)
plot(zipfTokens)

items: Vector[String] = Vector(
  "et",
...
counts: Vector[Int] = Vector(
  935,
...
res11_2: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
...
zipfTokens: Vector[Bar] = Vector(
  Bar(
...
res11_4: String = "plot-a724a8d6-2503-4b56-8f08-3d67519039ff"

# To work out

- relation of counts: 
    - lexical tokens in corpus
    - analyzed lexical tokens
    - recognized lexemes
- PoS distribution:  map each lexeme in lexeme histogram to its PoS 

(This is OK since lexical ambiguity is effectively 0)


## Histogram of disambiguated forms

In [14]:
val items = lc.formsHistogram.sorted.frequencies.map(fr => fr.item)
val counts = lc.formsHistogram.sorted.frequencies.map(fr => fr.count)
lc.labelledLexemeHistogram
val zipfForms = Vector(
  Bar(x = items, y = counts)
)
plot(zipfForms)

items: Vector[String] = Vector(
  "uninflected form: conjunction (ls.n16278)",
...
counts: Vector[Int] = Vector(
  923,
...
res13_2: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
...
zipfForms: Vector[Bar] = Vector(
  Bar(
...
res13_4: String = "plot-ebb3607c-2371-41e1-9ca6-3fb86f6c57fb"

## Create a map of lexeme to  PoS


In [19]:
val sampleForm = lc.analyzed.map (a => a.analyses(0))
val lexemePoSpairing = sampleForm.map (f => f.lemmaId -> f.posLabel)
val lexemeToPosMap = lexemePoSpairing.toMap

sampleForm: Vector[edu.holycross.shot.tabulae.LemmatizedForm] = Vector(
  IndeclinableForm(
...
lexemePoSpairing: Vector[(String, String)] = Vector(
  ("ls.n16519", "indeclinable"),
...
lexemeToPosMap: Map[String, String] = Map(
  "ls.n16519" -> "indeclinable",
...

In [20]:
val example = "ls.n16278"
lexemeToPosMap(example)

example: String = "ls.n16278"
res19_1: String = "indeclinable"

## Map lexeme histogram to PoS histogram

In [21]:
val freqOpts = lc.lexemeHistogram.frequencies.map(
  fr => {
    if (lexemeToPosMap.contains(fr.item)) {
      Some(edu.holycross.shot.histoutils.Frequency(lexemeToPosMap(fr.item),  fr.count))
    } else {
      None
    }
    
  })
val freqs = freqOpts.flatten

freqOpts: Vector[Option[edu.holycross.shot.histoutils.Frequency[String]]] = Vector(
  Some(Frequency("pronoun", 1136)),
...
freqs: Vector[edu.holycross.shot.histoutils.Frequency[String]] = Vector(
  Frequency("pronoun", 1136),
...